In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
import plotly.graph_objects as go

ecu = pd.read_csv(
    '~/Downloads/Conferences/Combined_CSV/AM_ath/errors/eastcarolinacombined.csv',
    engine='python',
    on_bad_lines='skip'
)


In [2]:
import plotly.graph_objects as go

fair_data = ecu[
    (ecu['Direction'] >= -45) &
    (ecu['Direction'] <= 45) &
     (ecu['Distance'] >= 30)
].dropna(subset=['Distance'])

fair_data['Direction_rad'] = np.deg2rad(fair_data['Direction'])
fair_data['SprayX'] = fair_data['Distance'] * np.sin(fair_data['Direction_rad'])
fair_data['SprayY'] = fair_data['Distance'] * np.cos(fair_data['Direction_rad'])


selected_batters = ['Zayac, Bradley', 'Irby, Austin', 'Lillis, David', 'White Jr., Ian']
subset_data = fair_data[fair_data['Batter'].isin(selected_batters)]
batters = subset_data['Batter'].unique()


# Initialize figure
fig = go.Figure()

# Add a trace for each batter, initially visible=False except the first
for i, batter in enumerate(batters):
    batter_data = fair_data[fair_data['Batter'] == batter]
    fig.add_trace(go.Scatter(
        x=batter_data['SprayX'],
        y=batter_data['SprayY'],
        mode='markers',
        marker=dict(size=8, color='blue', line=dict(width=1, color='black')),
        name=batter,
        visible=(i == 0),
        hovertemplate=(
            f"Batter: {batter}<br>" +
            "Distance: %{customdata[0]} ft<br>" +
            "Pitch Type: %{customdata[1]}"
        ),
        customdata=np.stack((batter_data['Distance'], batter_data['TaggedPitchType']), axis=-1)
    ))

# Dropdown menu
dropdown_buttons = [
    dict(
        label=batter,
        method='update',
        args=[
            {"visible": [i == j for j in range(len(batters))]},
            {"title": f"Spray Chart for {batter}"}
        ]
    )
    for i, batter in enumerate(batters)
]

# Foul lines as shapes
foul_lines = [
    dict(type="line", x0=0, y0=0, x1=-250, y1=250, line=dict(color="red", width=2)),
    dict(type="line", x0=0, y0=0, x1=250, y1=250, line=dict(color="red", width=2))
]

# Layout update
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_buttons,
            x=0.05,
            xanchor='left',
            y=1.15,
            yanchor='top'
        )
    ],
    shapes=foul_lines,
    title=f"Spray Chart",
    xaxis=dict(range=[-250, 250], title='Horizontal Distance (ft)'),
    yaxis=dict(range=[0, 400], title='Vertical Distance (ft)', scaleanchor='x'),
    plot_bgcolor='white',
    width=900,
    height=700
)

fig.show()
